In [1]:
#importar libs e variaveis/listas
import tableauserverclient as TSC
import pandas as pd
import pantab as pt
import zipfile
import re
import os
import shutil
from google.cloud import storage,bigquery
import pandas_gbq
import tableauhyperapi as thyapi


#funciona só no WINDOWS com as devidas chaves escondidas no arquivo ".env"
from dotenv import load_dotenv
load_dotenv()
#####################################


##############TABLEAU SERVER##################################################################
#chaves do tableau server
USERNAME = os.environ["USUARIO"]
PASSWORD = os.environ["SENHA"]
SITE_NAME =  os.environ["SITE_NAME"]

#chaves e nomes do datasource
ID_datasource= ["4e2c1d2d-2a94-43a6-8b66-43dcd500c0e9"]
NAME_datasource = ["Seguranca_de_linha"]
#############################################################################################

##############ARQUIVO DE LOGION GCP#########################################################

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ServiceKey_GoogleCloud.json'

#############################################################################################

##############GOOLE CLOUD STORAGE############################################################
#nome do bucket no Google Cloud Storage
#NAME_bucket = "data_teste_bucket"

#arquivo em PARQUET dataframe pandas
#NAME_dataset = NAME_datasource[0]+".parquet"
#############################################################################################

##############!!GCP - BIG QUERY!!!###########################################################

#ID do Projecto no Big query
ID_PROJECT_BQ = "tableau-api-378220"

#Nome do conjunto de dados e tabela do BQ
DATASET = "Teste"
Table1 = NAME_datasource[0]

DATASET_DATANAME = DATASET+"."+Table1

print("Iniciando software")
print(".......")
print(".......")
print(".......")
print("Variaveis e credenciais configuradas")
print(".......")
print(".......")
print(".......")

Iniciando software
.......
.......
.......
Variaveis e credenciais configuradas
.......
.......
.......


In [2]:
#login tableau server
tableau_auth = TSC.TableauAuth(USERNAME, PASSWORD, '')
server = TSC.Server(SITE_NAME)
server.use_highest_version()
print("Realizando login no Tableau server!")

Realizando login no Tableau server!


In [4]:
#baixar extrações 
print("Iniciando download dos arquivos")
with server.auth.sign_in(tableau_auth):
     for ID in range(0,len(ID_datasource)):
          while zipfile.is_zipfile(NAME_datasource[ID]+'.tdsx') ==False:
               server.datasources.download(datasource_id= ID_datasource[ID], include_extract=True)
               if zipfile.is_zipfile(NAME_datasource[ID]+'.tdsx') ==False:
                    print("Arquivo corrompido... baixando novamente...")
               else:
                    print("Arquivo baixado com sucesso!")


Iniciando download dos arquivos


In [ ]:
#extrair arquivos e extrair para Hyper na pasta "data"
for NAME in NAME_datasource:
    with zipfile.ZipFile(NAME +'.tdsx', 'r') as zip_ref:
        zip_ref.extractall('')

print("Arquivos extraidos")

In [ ]:
with thyapi.HyperProcess(telemetry=thyapi.Telemetry.DO_NOT_SEND_USAGE_DATA_TO_TABLEAU ) as hyper:
    with thyapi.Connection(endpoint=hyper.endpoint, database='Data\Extracts\Seguranca_de_linha.hyper') as connection:
         table_names = connection.catalog.get_table_names(schema="Extract")
         for table in table_names:
            table_definition = connection.catalog.get_table_definition(name=table)
            print(f"Table {table.name} has qualified name: {table}")
  
            for column in table_definition.columns:
                print(f"Column {column.name} has type={column.type} and nullability={column.nullability}")

            print("")
            # Print all rows from the "Extract"."Extract" table.
            table_name = thyapi.TableName("Extract", "Extract")
            ###print(f"These are all rows in the table {table_name}:")
            # `execute_list_query` executes a SQL query and returns the result as list of rows of data,
            # each represented by a list of objects.

            max_rows = connection.execute_list_query(query=f"SELECT COUNT(*) FROM {table_name}")
            print(f"This table has {max_rows[0][0]} rows " )
            ###print("The connection to the Hyper file has been closed.")
            rows_in_list= connection.execute_list_query(query=f"SELECT * FROM {table_name} LIMIT 1000")
    

In [ ]:
#carrega os datasets, conversão de .hyper para dataset pandas
dataset = pt.frame_from_hyper('Data/Extracts/'+ NAME_datasource[0] +'.hyper',table=table_name,use_float_na=True )
dataset.to_csv("teste_base_seguranca_linha.csv", index=False)


In [ ]:
#tratamento de excessão pois não aceita "espaços" como nome de coluna
list_columns = dataset.columns
list_columns = [re.sub(r' ', '_', file) for file in list_columns]
dataset.columns = list_columns

In [ ]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
client = bigquery.Client(project='self-service-saude')

In [ ]:
#deleta todos os arquivos [tds,tdsx] e pastas da extração
shutil.rmtree('Data')
for arquivo in range (0, len(NAME_datasource)):
    os.remove(NAME_datasource[arquivo]+'.tdsx')
    os.remove(NAME_datasource[arquivo]+'.tds')

In [ ]:
#autenticação GOOGLE CLOUD
#storage_client = storage.Client()

#autenticação no bucket escolhido do Google Cloud Storage e BigQuery
#my_bucket = storage_client.get_bucket(NAME_bucket) 
bigquery_client = bigquery.Client()

In [ ]:
######função para subir arquivo no Google Cloud Storage
#def upload_bucket(blob_name, file_path, bucket_name):
#    try:
#        bucket = storage_client.get_bucket(bucket_name)
#        blob = bucket.blob(blob_name)
#        blob.upload_from_filename(file_path)
#        return True
#    except Exception as e:
#        print(e)
#        return False
######Função para download de arquivo no Goole Cloud Storage    
#def download_file_bucket(blob_name, file_path, bucket_name):
#    try:
#        bucket = storage_client.get_bucket(bucket_name)
#        blob = bucket.blob(blob_name)
#        with open(file_path, 'wb') as f:
#            storage_client.download_blob_to_file(blob,f)
#        return True
#    except Exception as e:
#        print(e)
#        return False

#### Download Exemplo
# download_file_bucket(NAME_dataset,os.path.join(os.getcwd(),NAME_dataset),NAME_bucket)

In [ ]:
#upload dos arquivos ao Bucket
#upload_bucket(NAME_dataset,os.path.abspath(NAME_dataset),NAME_bucket)

In [ ]:
#remoção  dos arquivos pandas em .parquet
#os.remove(NAME_dataset)

In [ ]:
#função para subir o dataset pandas direto para o Bigquery
def load_bq(df,dataset):
    pandas_gbq.to_gbq(
        df, dataset,project_id =ID_PROJECT_BQ,if_exists="replace"
        )
    

In [ ]:
#upload das extrações para o BigQUERY
load_bq(dataset,DATASET_DATANAME)

